# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [3]:
pip install folium

Note: you may need to restart the kernel to use updated packages.


In [8]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests
import folium 
# Import API key
from api_keys import geoapify_key

In [5]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,faya,18.3851,42.4509,22.06,35,21,2.60,SA,1666108228
1,1,farsund,58.0948,6.8047,13.30,100,0,7.65,NO,1666108228
2,2,new norfolk,-42.7826,147.0587,11.72,58,12,1.34,AU,1666108230
3,3,jamestown,42.0970,-79.2353,5.77,77,100,9.77,US,1666107934
4,4,lanzhou,36.0564,103.7922,14.53,48,59,1.20,CN,1666108230


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [6]:
%%capture --no-display

# Create a map centered around the average latitude and longitude of the cities
map_center = [city_data_df['Lat'].mean(), city_data_df['Lng'].mean()]
city_map = folium.Map(location=map_center, zoom_start=5)

# Add markers for each city in the DataFrame
for index, row in city_data_df.iterrows():
    folium.Marker(
        location=[row['Lat'], row['Lng']],
        popup=f"{row['City']}, {row['Country']}",
        icon=folium.Icon(color='blue')
    ).add_to(city_map)

# Display the map
city_map

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [9]:
# Narrow down cities that fit criteria and drop any results with null values
ideal_cities_df = city_data_df[
    (city_data_df['Max Temp'] >= 70) & 
    (city_data_df['Max Temp'] <= 80) & 
    (city_data_df['Wind Speed'] <= 10) & 
    (city_data_df['Cloudiness'] == 0)
]

# Drop any rows with null values
ideal_cities_df = ideal_cities_df.dropna()

# Display sample data
ideal_cities_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date


### Step 3: Create a new DataFrame called `hotel_df`.

In [10]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = city_data_df[['City', 'Country', 'Lat', 'Lng', 'Humidity']].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df['Hotel Name'] = ""

# Display sample data
hotel_df.head()

,City,Country,Lat,Lng,Humidity,Hotel Name
0,faya,SA,18.3851,42.4509,35,
1,farsund,NO,58.0948,6.8047,100,
2,new norfolk,AU,-42.7826,147.0587,58,
3,jamestown,US,42.0970,-79.2353,77,
4,lanzhou,CN,36.0564,103.7922,48,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [12]:

# Set parameters to search for a hotel
radius = 10000  # 10 km radius
params = {
    "categories": "accommodation.hotel",
    "apiKey": "YOUR_GEOAPIFY_API_KEY",
    "limit": 1
}


# Print a message to follow up the hotel search
print("Starting hotel search")


# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    lat = row['Lat']
    lng = row['Lng']

    # Add the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{lng},{lat},{radius}"
    params["bias"] = f"proximity:{lng},{lat}"

    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"

    # Make an API request using the params dictionary
    response = requests.get(base_url, params=params)

    # Convert the API response to JSON format
    name_address = response.json()

    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"

    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df.head()

Starting hotel search
faya - nearest hotel: No hotel found
farsund - nearest hotel: No hotel found
new norfolk - nearest hotel: No hotel found
jamestown - nearest hotel: No hotel found
lanzhou - nearest hotel: No hotel found
ushuaia - nearest hotel: No hotel found
albany - nearest hotel: No hotel found
carnarvon - nearest hotel: No hotel found
sisimiut - nearest hotel: No hotel found
saint-pierre - nearest hotel: No hotel found
colorado - nearest hotel: No hotel found
nikolskoye - nearest hotel: No hotel found
trairi - nearest hotel: No hotel found
dobryanka - nearest hotel: No hotel found
qaanaaq - nearest hotel: No hotel found
mataura - nearest hotel: No hotel found
khatanga - nearest hotel: No hotel found
tasiilaq - nearest hotel: No hotel found
tiksi - nearest hotel: No hotel found
puerto ayora - nearest hotel: No hotel found
avarua - nearest hotel: No hotel found
nuevo laredo - nearest hotel: No hotel found
guiratinga - nearest hotel: No hotel found
rikitea - nearest hotel: No hot

,City,Country,Lat,Lng,Humidity,Hotel Name
0,faya,SA,18.3851,42.4509,35,No hotel found
1,farsund,NO,58.0948,6.8047,100,No hotel found
2,new norfolk,AU,-42.7826,147.0587,58,No hotel found
3,jamestown,US,42.0970,-79.2353,77,No hotel found
4,lanzhou,CN,36.0564,103.7922,48,No hotel found


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [13]:
%%capture --no-display

# Create a map centered around the average latitude and longitude of the cities
map_center = [city_data_df['Lat'].mean(), city_data_df['Lng'].mean()]
city_map = folium.Map(location=map_center, zoom_start=5)

# Add markers for each city in the DataFrame
for index, row in city_data_df.iterrows():
    folium.Marker(
        location=[row['Lat'], row['Lng']],
        popup=f"{row['City']}, {row['Country']}",
        icon=folium.Icon(color='blue')
    ).add_to(city_map)

# Display the map
city_map